In [1]:
!pip install scikit-learn-intelex

In [2]:
from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
# math related operations
import numpy as np
import pandas as pd

# machine learning
import cv2
import os
from random import shuffle
from tqdm import tqdm
import random

# to open, load, convert array to image
from PIL import Image

# to preprocess the image
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Since it is multiclass classification, one hot encoding is needed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# for augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# MobileNetV3 model
from tensorflow.keras.applications import mobilenet_v3
from tensorflow.keras import Model, layers
from numpy import loadtxt

import itertools
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image